In [13]:
# Initializing the "people" class
class People:
    def __init__(self, startFloor: int, destinationFloor: int, amount: int = 1):
        self.startFloor = startFloor
        self.destinationFloor = destinationFloor
        self.amount = amount  # Amount of people that's in a group
        self.waitingTimer = 0  # Wait time
        self.journeyTimer = 0  # Journey time
        self.direction = True if (destinationFloor - startFloor > 0) else False
        self.elevatorBreakCount = (
            0  # The amount of pauses before reaching the destination
        )


# Lists of people that are already out. Will not be updated when left elevator
class Outlist:
    def __init__(self, outlist: list = []):
        self.outlist = outlist

    # For adding people into outlist.
    def addOutlist(self, *args: People):
        for i in args:
            self.outlist.append(i)


# Initializing the Elevator class
class Elevator:
    def __init__(
        self,
        direction: bool,
        floor: int,
        minFloor: int,
        maxFloor: int,
        carryLimit: int,
        bufferCutoff: float,
        outList: Outlist,
        progression: float = 0,
        activeSpeed: float = 5,
        bufferSpeed: float = 1,
        *args: People
    ):
        # Properties of elevator
        self.direction = direction  # True -> going up, False = going down
        self.floor = floor
        self.minFloor = minFloor
        self.maxFloor = maxFloor
        self.carryLimit = carryLimit
        self.progression = (
            # In the range 0 - 100 (If enters new floor, progression = 0)
            progression
        )
        self.bufferCutoff = bufferCutoff  # In the range 0 - 100
        self.activeSpeed = activeSpeed
        self.bufferSpeed = bufferSpeed

        # Queues
        self.internalQueue = []
        self.internalQueueNum = len(self.internalQueue)
        self.externalQueue = []
        self.outList = outList
        for i in args:
            self.peopleIn.append(i)

    # Adding people to external queue
    def addExternalQueue(self, *args: People):
        for i in args:
            self.externalQueue.append(i)

    # Adding people to internal queue
    def addInternalQueue(self, *args: People):
        for i in args:
            self.internalQueue.append(i)
        self.internalQueueNum += len(args)

    # Unload people from internal queue to outlist
    # Checks every people in internal queue. If the destination floor matches the elevator's current floor, move the people to the outlist
    def unloadFromInternalQueue(self):
        # Creating a boolean list. True if the destination floor matches the current floor, false otherwise.
        outQueueBool = [i.destinationFloor == self.floor for i in self.internalQueue]
        internalTemp = []
        # If true, move to outlist, pass otherwise.
        for people, boolean in zip(self.internalQueue, outQueueBool):
            if boolean:
                self.outList.addOutlist(people)
            else:
                internalTemp.append(people)
        self.internalQueue = internalTemp

    # Load people from external queue to internal queue
    # Checks every people in external queue. Pop if possible.
    ### Will be implemented
    def loadFromExternalQueue(self):
        # Creating a boolean list. True if the current floor matches the starting floor, false otherwise.
        inQueueBool = [i.destinationFloor == self.floor for i in self.externalQueue]
        self.internalQueueNum = len(self.internalQueue)
        freeSpaceNum = self.carryLimit - self.internalQueueNum
        # If true, move to temporary queues
        inQueueTemp = []
        exQueueTemp = []
        for people, boolean in zip(self.externalQueue, inQueueBool):
            if boolean:
                inQueueTemp.append(people)
            else:
                exQueueTemp.append(people)
        # Finding the cumulative sum of people
        cumulativeSum = 0
        for people in inQueueTemp:
            if cumulativeSum > freeSpaceNum:
                break
            else:
                cumulativeSum += people.amount
                inQueueTemp.append(people)
                exQueueTemp.pop(index = 1)
        self.internalQueue.extend(inQueueTemp)
        self.externalQueue = exQueueTemp

    # Progressing the elevator
    def progressElevator(self):
        if self.floor >= self.maxFloor:
            self.direction = not self.direction
            self.floor -= 1
            self.progression = 0
        if self.floor <= self.minFloor:
            self.direction = not self.direction
            self.floor += 1
            self.progression = 0

        if self.progression >= self.bufferCutoff:
            speed = self.bufferSpeed
        else:
            speed = self.activeSpeed

        self.progression += speed
        if self.progression > 100:
            self.progression = 0
            if self.direction == True:
                self.floor += 1
            else:
                self.floor -= 1

    # def directionalScan(self):
    #     internalQueueFiltered = self.internalQueue.sort(key = lambda person: person.amount)
    #     externalQueueFiltered = filter(lambda person: person.direction == self.direction, self.externalQueue)

In [8]:
testOutlist = Outlist()
testElevator = Elevator(
    direction=False,
    floor=2,
    minFloor=1,
    maxFloor=3,
    carryLimit=10,
    bufferCutoff=60,
    progression=99,
    outList=testOutlist,
)
internalTemp = [People(1, 3), People(3, 2), People(5, 2)]
externalTemp = [People(6, 4), People(2, 3), People(1, 4)]
testElevator.addInternalQueue(*internalTemp)
testElevator.addExternalQueue(*externalTemp)